# LAB - FANN

With this file you will create a dataset to train a neural network and then load convert the model to a suitable one for your microcontroller.

You have to create a dataset for **at least 3 letters** and then train and validate FANN with it. You have
to record each letter **at least 30 times** for the training set, and **at least 10 times** for the validation set.

Execute the cell below only one time so that you can install the missing package. If you are working on a K63's computer than you don't need to run it.


In [ ]:
import sys
!{sys.executable} -m pip install pyserial

You will now connect with your microcontroller via UART. Please just put the COM number (e.g if "COM8" than just write "8")

In [ ]:
import serial.tools.list_ports
print('Com ports list:')
comPorts = list(serial.tools.list_ports.comports())
for comPort in comPorts:
    print(comPort)
chooseComPort = input('Please insert port number: ')
ser = serial.Serial('COM' + chooseComPort, 115200)

In [ ]:
def take_data_func(data_to_write):
    for element in data_to_write:
        file_to_open.write(element + ' ')
    file_to_open.write('\n' + output + '\n')
    return


def convert_to_list(value):
    value = value.replace("b' ", "")
    return value.split(", ")

Type in the letter you want to record (append **"_train"** for training set or **"_val"** for the validation set) and assign it a label. <br>
**E.g.:** 3 letters: S_train -> 1 0 0, M_train -> 0 1 0, X_train -> 0 0 1, where 1 0 0, 0 1 0, 0 0 1 are the corresponding labels.

Create first the training set for all letters and then the validation set.

**IMPORTANT:** put a space between each number of the label.

In [ ]:
letter = input('Please insert letter to collect data: ')
output = input('Please insert output (e.g. 1 0 0 0): ')
file_to_open = open('Samples/' + letter + '.dat', 'a')

sampleNumber = 1

while input('1 - acquire sample, 2 - exit: ') == '1':
    print('Sample number: ' + str(sampleNumber))
    line = ser.readline()
    lineList = convert_to_list(str(line))
    del lineList[-1]
    print(lineList)
    print('Collected ' + str(len(lineList)) + ' data')
    take_data_func(lineList)
    sampleNumber = sampleNumber + 1

print('Close file')
file_to_open.close()

### Go to the next step only if you alredy collected data for all 3 letters

After you collected all the data generate the final datasets for your neural network.

1 - Validation Dataset <br>
2 - Training Dataset

You will find the result in the "Dataset" folder

In [ ]:
import glob
import random
import os
import sys

data = list()
test = list()

os.chdir("Samples")

datasetVersion = input("Do you want to create: 1 - validation dataset, 2 - training dataset")

if(datasetVersion == '1'):
    dataToSearch = '*_val.dat'
else:
    dataToSearch = '*_train.dat'

for filename in glob.glob(dataToSearch):
    file = open(filename)
    lines = file.readlines()
    for i in range(0, len(lines), 2):
        data.append(lines[i:i + 2])
        test.append(lines[i:i + 2])
    file.close()

random.shuffle(data)

length = len(data)
lengthInput = len(data[0][1].strip().replace(' ', ''))
folderName = 'Dataset'

os.chdir("..")
if os.path.isdir(folderName) == False:
    os.makedirs(folderName)

filename = input('Please insert the file name: ')
f = open(folderName + '/' + filename + '.dat', 'w+')
f.write('%i %i %i\n' % (length, 90, lengthInput))
for i in range(len(data)):
    for j in range(len(data[i])):
        f.write(data[i][j])

f.close()

### Go to the next step only if you alredy created your final dataset

This script will generate the library you have to load in your project. <br>
You will find all the files in the "STMFANN" folder.

In [ ]:
from math import log

fname = input("Insert the name of the File without extension: ")

try:
    netF = open("Dataset/" + fname + ".net", 'r')
    # find out whether it's a fixed or float version
    fann = {}
    firstL = netF.readline()
    fann["nettype"] = "float"
    if "FIX" in firstL:
        print("nettype: fixed")
        fann["nettype"] = "int"
    else:
        print("nettype: float")


    file = netF.readlines()
    for line in file:
        parts = line.strip('\n').split('=')

        # if it is not split then we have an invalid line without an '='
        if(len(parts) != 2):
            continue

        # neuron and connection data have specifiers we need to remove
        if(len(parts[0].split(" ")) > 1):
            parts[0] = parts[0].split(" ")[0]
        
        # store all variables in dictionary
        fann[parts[0]] = parts[1]

    # currently no networks with a connection_rate below 1 are supported
    if float(fann["connection_rate"]) < 1.0:
        print("Currently no networks with a connection_rate below 1.0 are supported")
        exit(-1)

    # reformat neurons and connections
    fann["neurons"] = fann["neurons"].strip("()\r\n ").split(") (")
    fann["connections"] = fann["connections"].strip("()\r\n ").split(") (")

    tot_connections = 0
    fann["generated_neurons"] = []
    for neuron in fann["neurons"]:
        neuron = neuron.split(', ')

        num_inputs = neuron[0]
        activation_function = neuron[1]
        activation_steepness = neuron[2]

        first_connection = tot_connections
        last_connection = first_connection + int(num_inputs)

        fann["generated_neurons"].append("{" + ', '.join((str(first_connection), str(last_connection), activation_steepness, activation_function)) + "}")

        tot_connections = last_connection

    if tot_connections != len(fann["connections"]):
        print("ERROR: tot_connections != len(connections)")
        print(tot_connections)
        print(len(fann["connections"]))
        exit(-1)

    fann["generated_connections"] = []
    for connection in fann["connections"]:
        connection = connection.split(", ")
        fann["generated_connections"].append(connection[1])

    fann["generated_layers"] = []
    layer_num = 0
    fann['layer_sizes'] = fann['layer_sizes'].strip()
    for layer in fann['layer_sizes'].split(' '):
        fann["generated_layers"].append('{' + str(layer_num) + ', ' + str(layer_num + int(layer)) + '}')
        layer_num = layer_num + int(layer)

    if "decimal_point" not in fann:
        fann['decimal_point'] = "1"

    fann["multiplier"] = 1 << int(fann["decimal_point"])
        
    fann["num_input"] = str(int(fann['layer_sizes'].split(' ')[0]) - 1)
    fann["num_output"] = str(int(fann['layer_sizes'].strip(' ').split(' ')[-1]) - 1)

    # calculate sigmoid functions
    multiplier = int(fann["multiplier"])
    precalc_fixed = {}
    precalc_fixed["SIGMOID_RESULTS_0"] = max(int(multiplier / 200.0 + 0.5), 1)
    precalc_fixed["SIGMOID_RESULTS_1"] = max(int(multiplier / 20.0 + 0.5), 1)
    precalc_fixed["SIGMOID_RESULTS_2"] = max(int(multiplier / 4.0 + 0.5), 1)
    precalc_fixed["SIGMOID_RESULTS_3"] = min(multiplier - int(multiplier / 4.0 + 0.5), multiplier - 1)
    precalc_fixed["SIGMOID_RESULTS_4"] = min(multiplier - int(multiplier / 20.0 + 0.5), multiplier - 1)
    precalc_fixed["SIGMOID_RESULTS_5"] = min(multiplier - int(multiplier / 200.0 + 0.5), multiplier - 1)

    precalc_fixed["SIGMOID_SYMMETRIC_RESULTS_0"] = max(int((multiplier / 100.0) - multiplier - 0.5), int(1 - multiplier))
    precalc_fixed["SIGMOID_SYMMETRIC_RESULTS_1"] = max(int((multiplier / 10.0) - multiplier - 0.5), int(1 - multiplier))
    precalc_fixed["SIGMOID_SYMMETRIC_RESULTS_2"] = max(int((multiplier / 2.0) - multiplier - 0.5), int(1 - multiplier))
    precalc_fixed["SIGMOID_SYMMETRIC_RESULTS_3"] = min(multiplier - int(multiplier / 2.0 + 0.5), multiplier - 1)
    precalc_fixed["SIGMOID_SYMMETRIC_RESULTS_4"] = min(multiplier - int(multiplier / 10.0 + 0.5), multiplier - 1)
    precalc_fixed["SIGMOID_SYMMETRIC_RESULTS_5"] = min(multiplier - int(multiplier / 100.0 + 0.5), multiplier - 1)

    for i in range(0, 6):
        precalc_fixed["SIGMOID_VALUES_" + str(i)] = int(((log(multiplier / float(precalc_fixed["SIGMOID_RESULTS_" + str(i)]) - 1) * float(multiplier)) / -2.0) * float(multiplier))
        precalc_fixed["SIGMOID_SYMMETRIC_VALUES_" + str(i)] = int(((log((multiplier - float(precalc_fixed["SIGMOID_SYMMETRIC_RESULTS_" + str(i)])) / float(precalc_fixed["SIGMOID_SYMMETRIC_RESULTS_" + str(i)] + multiplier)) * float(multiplier)) / -2.0) * float(multiplier))

    # generate file contents for fann_conf.h
    saveString = '#ifndef FANN_FANN_CONF_H_\n'
    saveString = saveString + '#define FANN_FANN_CONF_H_\n\n'

    if fann["nettype"] == "int":
        saveString = saveString + '#define FIXEDFANN\n\n'

    for x in precalc_fixed:
        saveString = saveString + '#define ' + x + ' ' + str(precalc_fixed[x]) + '\n'

    saveString = saveString + '\n#define NUM_NEURONS ' + str(len(fann["generated_neurons"])) + '\n'
    saveString = saveString + '#define MULTIPLIER ' + str(fann["multiplier"]) + '\n'
    saveString = saveString + '#define DECIMAL_POINT ' + fann["decimal_point"] + '\n'
    saveString = saveString + '#define NUM_INPUT ' + fann["num_input"] + '\n'
    saveString = saveString + '#define NUM_OUTPUT ' + fann["num_output"] + '\n'
    saveString = saveString + '#define NUM_LAYERS ' + str(len(fann["generated_layers"])) + '\n'
    saveString = saveString + '#define CONNECTION_RATE ' + fann["connection_rate"] + '\n\n'

    saveString = saveString + '\n#endif // FANN_FANN_CONF_H_\n'

    try:
        FW = open('STMFANN/fann_conf.h', "w")
        FW.write(saveString)
        FW.close()
    except IOError:
        print("Could not open write fann_conf.h")
        exit(1)

    # generate file contents for fann_net.h
    saveString = '#ifndef FANN_FANN_NET_H_\n'
    saveString = saveString + '#define FANN_FANN_NET_H_\n\n'

    #insert includes
    saveString = saveString + '#include "fann.h" \n'
    saveString = saveString + '#include "fann_structs.h" \n\n'
    
    saveString = saveString + 'const enum fann_nettype_enum network_type = ' + fann["network_type"] + ';\n\n'
    saveString = saveString + 'const fann_neuron fann_neurons[' + str(len(fann["generated_neurons"])) + '] = {' + ', '.join(fann["generated_neurons"]) + '};\n\n'
    saveString = saveString + 'fann_type fann_weights[' + str(len(fann["generated_connections"])) + '] = {' + ', '.join(fann["generated_connections"]) + '};\n\n'
    saveString = saveString + 'const fann_layer fann_layers[' + str(len(fann["generated_layers"])) + '] = {' + ', '.join(fann["generated_layers"]) + '};\n\n'

    saveString = saveString + '\n#endif // FANN_FANN_NET_H_\n'

    try:
        FW = open('STMFANN/fann_net.h', "w")
        FW.write(saveString)
        FW.close()
    except IOError:
        print("Could not open write fann_net.h")
        exit(1)

    print("generated fann_net.h")
    print("generated fann_conf.h")
    print("NETWORK converted. Please copy the files and/or recompile")
except IOError:
    print("Could not open " + fname + ".net")
    print("Failed to generate network from file")